### Approach #1 (Wrong)

In [2]:
import jet
import numpy as np

In [2]:
# Defining Hadamard gate
inv_sqrt_2 = 2 ** -0.5
# The Hadamard gate can be viewed as a rank-2 tensor of size 2 by 2
H = jet.Tensor(["i", "k"], [2, 2], [inv_sqrt_2, inv_sqrt_2, inv_sqrt_2, -inv_sqrt_2])

In [3]:
# Defining CNOT gate
# We can view it as a rank-4 tensor of size 2 by 2 by 2 by 2
# Two of the legs of this tensor represent inputs and 
# the other two legs represent outputs
CNOT = jet.Tensor(["k", "j", "l", "m"], [2, 2, 2, 2])
CNOT.set_value((0, 0, 0, 0), 1) # |00> -> |00>
CNOT.set_value((0, 1, 0, 1), 1) # |01> -> |01>
CNOT.set_value((1, 0, 1, 1), 1) # |10> -> |11>
CNOT.set_value((1, 1, 1, 0), 1) # |11> -> |10>

In [4]:
# Defining Pauli-Z Gate
pauli_z_data = [1, 0, 0, -1]
size = [2,2]
indices = ["l","n"]
Z = jet.Tensor(indices, size, pauli_z_data)

In [5]:
# Defining Hadamard gate
inv_sqrt_2 = 2 ** -0.5
# The Hadamard gate can be viewed as a rank-2 tensor of size 2 by 2
new_H = jet.Tensor(["r", "t"], [2, 2], [inv_sqrt_2, inv_sqrt_2, inv_sqrt_2, -inv_sqrt_2])

In [6]:
# Defining CNOT gate
# We can view it as a rank-4 tensor of size 2 by 2 by 2 by 2
# Two of the legs of this tensor represent inputs and 
# the other two legs represent outputs
new_CNOT = jet.Tensor(["n", "m", "r", "s"], [2, 2, 2, 2])
new_CNOT.set_value((0, 0, 0, 0), 1) # |00> -> |00>
new_CNOT.set_value((0, 1, 0, 1), 1) # |01> -> |01>
new_CNOT.set_value((1, 0, 1, 1), 1) # |10> -> |11>
new_CNOT.set_value((1, 1, 1, 0), 1) # |11> -> |10>

In [7]:
# Explicitly define tensors with indices to mimic quantum circuit
# Contract network to get resulting tensorproduct of paulis
tn = jet.TensorNetwork()
tn.add_tensor(H)
tn.add_tensor(CNOT)
tn.add_tensor(Z)
tn.add_tensor(new_H)
tn.add_tensor(new_CNOT)
result = tn.contract()
result = np.reshape(result.data, [4,4])
print(result)



[[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]]


The above matrix is equal to $X \otimes I$ (Labeling of indices seems a little wonky right now, still trying to figure this out Edit: The above result is wrong as labeling of indices is wrong)

### Approach #2

In [8]:
# Let's try implementing above tensor network using Jet's natively defined gates
tn = jet.TensorNetwork()
CNOT_tr = jet.CX().tensor()
H_tr = jet.Hadamard().tensor()
H_tr.rename_index(0, '2')
H_tr.rename_index(1, '4')
Z = jet.PauliZ().tensor()
Z.rename_index(0, '4')
Z.rename_index(1, '5')
H = jet.Hadamard().tensor()
H.rename_index(0, '5')
H.rename_index(1, '6')
CNOT = jet.CX().tensor()
CNOT.rename_index(0, '6')
CNOT.rename_index(1, '3')
CNOT.rename_index(2, '7')
CNOT.rename_index(3, '8')

tn.add_tensor(CNOT_tr)
tn.add_tensor(H_tr)
tn.add_tensor(Z)
tn.add_tensor(H)
tn.add_tensor(CNOT)
result = tn.contract()
result = np.reshape(result.data, [4,4])
print(result)

[[0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]]


The above matrix is equal to $X \otimes X$ which is what we accept it to be

### Approach #3 (Defining our own gate)

In [6]:

from functools import lru_cache

INV_SQRT2 = 1 / np.sqrt(2)

class UGate(jet.QubitGate):
    """ UGate represents our QCSAT-instance"""

    def __init__(self):
        super().__init__(name="UGate", num_wires = 2)

    @lru_cache()

    def _data(self):
        mat1 = np.kron(np.array([[INV_SQRT2, INV_SQRT2], [INV_SQRT2, -INV_SQRT2]]), np.array([[1,0],[0,1]]))
        mat2 = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
        mat = np.matmul(mat1,mat2)
        return mat
    

In [9]:
tn = jet.TensorNetwork()
U_tr = jet.Adjoint(UGate()).tensor()
U_tr.rename_index(0, '0')
U_tr.rename_index(1, '1')
U_tr.rename_index(2, '4')
U_tr.rename_index(3, '3')
U = UGate().tensor()
U.rename_index(0, '5')
U.rename_index(1, '3')
U.rename_index(2, '7')
U.rename_index(3, '8')
Z = jet.PauliZ().tensor()
Z.rename_index(0, '4')
Z.rename_index(1, '5')
tn.add_tensor(U_tr)
tn.add_tensor(Z)
tn.add_tensor(U)
result = tn.contract()
result = np.reshape(result.data, [4,4])
print(result)


[[0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]]


By defining our own gate 'U', we can achieve the same, desired results of $X \otimes X$